In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
from tensorflow import keras
import json
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
import dataset
from util import (
    get_place_to_index_mapping,
    get_incident_to_index_mapping
)
from keras.applications.resnet50 import ResNet50
from keras.layers import Dense, Flatten, Permute
from keras import Sequential
import keras.backend as kb
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import PIL

In [3]:
abs_path = "/kuacc/users/asafaya19/cv-project"
train_json = os.path.join(abs_path ,"eccv_train.json")
val_json = os.path.join(abs_path ,"eccv_val.json")
data_dir = os.path.join(abs_path, "data")
train_dir = os.path.join(data_dir, "train")
val_dir = os.path.join(data_dir, "val")

train_paths = json.loads(open(train_json).readline())
val_paths = json.loads(open(val_json).readline())

place_to_idx = get_place_to_index_mapping()
incident_to_idx = get_incident_to_index_mapping()

In [4]:
def get_dataset(paths,file_dir, threshold=1000):
    train_set = []
    for path in tqdm(paths, leave=False):
    #     print(path)
        if not os.path.exists(os.path.join(file_dir, path)):
            continue
        # Make sure image is not corrupt, try importing it
        try:
            PIL.Image.open(os.path.join(file_dir, path))
        except:
            continue
        nump = len(place_to_idx)+1
        numi = len(incident_to_idx)+1
        place_labels = np.zeros(nump, np.float32)
        place_weights = np.zeros(nump, np.float32)
        incident_labels = np.zeros(numi, np.float32)
        incident_weights = np.zeros(numi, np.float32)

        incidents = paths[path]["incidents"]
        for k in incidents:
            lbl = incidents[k]
            if lbl==1:
                # We are sure this instance is only this incident
                incident_labels[incident_to_idx[k]]=1
                incident_weights = np.ones(numi, np.float32)
            else:
                # We are only sure that this image is not that incident
                incident_weights[incident_to_idx[k]]=1
        if len(incidents)==0:
            # No incident
            incident_labels[-1]=1
            incident_weights = np.ones(numi, np.float32)

        places = paths[path]["places"]
        for k in places:
            lbl = places[k]
            if lbl==1:
                # We are sure this instance is only this incident
                place_labels[place_to_idx[k]]=1
                place_weights = np.ones(nump, np.float32)
            else:
                # We are only sure that this image is not that incident
                place_weights[place_to_idx[k]]=1
        if len(places)==0:
            # No place
            place_labels[-1]=1
            place_weights = np.ones(nump, np.float32)


        train_set.append({
            "path":path,
            "incident_labels":incident_labels,
            "incident_weights":incident_weights,
            "incidents":np.vstack((incident_labels, incident_weights)),
            "place_labels":place_labels,
            "place_weights":place_weights,
            "place":np.vstack((place_labels, place_weights))
        })
        if len(train_set)>=threshold:
            break
    return train_set

In [5]:
def getpreprocessfunc():
    mean = np.asarray([0.485, 0.456, 0.406]).reshape(3, 1, 1).astype(np.float32)
    std = np.asarray([0.229, 0.224, 0.225]).reshape(3, 1, 1).astype(np.float32)
    def preprocessfunc(img):
        img /= 255
        img -= mean
        img /= std
        return img
    return preprocessfunc

In [6]:

def get_final_model():
    
    
    inp = keras.Input(shape=(3, 256, 256))
#     cropped = keras.layers.experimental.preprocessing.RandomCrop(224, 224)(inp)
    
    loaded_model = model_from_json(open('models/trunk.json', 'r').read())(inp)
    loaded_model.load_weights("models/trunk.h5")

    incident_proj = Dense(len(incident_to_idx)+1, name="incidents_projection")(trunk)
    incident_proj.se("models")
    places_proj = Dense(len(place_to_idx)+1, name="places_projection")(trunk)
    
    mdl = keras.Model(inputs=inp, outputs=[incident_proj, places_proj])    
    
    mdl.layers[2].set_weights(np.load("models/incident.npy"))
    
    
    return mdl

In [7]:
# Enclosure to retain state
def get_weighted_accuracy():
    m = keras.metrics.CategoricalAccuracy()
    def weighted_accuracy(y_true, y_preds):
        y_true = tf.reshape(y_true, (bs, 2, -1))
        y_true_lbls = y_true[:,0,:]
        return m(y_true_lbls, y_preds)
    return weighted_accuracy

In [8]:
def weighted_loss(y_true, y_preds):
    bce = keras.losses.BinaryCrossentropy(keras.losses.Reduction.NONE)
#     tf.print(y_true)
#     tf.print(y_preds)
#     print(y_true.shape)
#     print(y_preds.shape)
    bs = y_true.shape[0]
    y_true = tf.reshape(y_true, (bs, 2, -1))
    y_true_lbls = y_true[:,0,:]
    y_true_weights = y_true[:,1,:]
    bce_loss = bce(y_true_lbls, y_preds)
#     import pdb
#     pdb.set_trace()
#     return bce_loss
    return tf.reduce_sum(tf.multiply(bce_loss, y_true_weights))

In [16]:
mdl = get_final_model()

In [6]:
incident_proj = Dense(len(incident_to_idx)+1, name="incidents_projection")

# 

In [15]:
from keras.models import model_from_json


# inp = keras.Input(shape=(3, 256, 256))
# permute = Permute((2, 3, 1))
# cropped = keras.layers.experimental.preprocessing.RandomCrop(224, 224)
# permuteback = Permute((3, 1, 2))
# loaded_model = model_from_json(open('models/trunk.json', 'r').read())
# loaded_model.load_weights("models/trunk.h5")

# # Define Sequential model with 3 layers
# trunk_model = keras.Sequential(
#     [
#         inp,
#         permute,
#         cropped,
#         permuteback,
#         loaded_model,
#     ]
# )

# incident_proj = Dense(len(incident_to_idx)+1, name="incidents_projection")(trunk_model)
# places_proj = Dense(len(place_to_idx)+1, name="places_projection")(trunk_model)

# mdl = keras.Model(inputs=trunk_model, outputs=[incident_proj, places_proj])    


class FinalModel(keras.Model):
    def __init__(self):
        super(FinalModel, self).__init__()
        self.inp = keras.Input(shape=(3, 256, 256))
        self.permute = Permute((2, 3, 1))
        self.cropped = keras.layers.experimental.preprocessing.RandomCrop(224, 224)
        self.permuteback = Permute((3, 1, 2))
        self.loaded_model = model_from_json(open(os.path.join(abs_path,'models/trunk.json'), 'r').read())
        self.loaded_model.load_weights(os.path.join(abs_path,"models/trunk.h5"))
        
    def call(self, inputs):
#         x = self.inp(inputs)
        x = self.permute(inputs)
        x = self.cropped(x)
        x = self.permuteback(x)
        x = self.loaded_model(x)
        
        return x

mdl = FinalModel()

In [24]:
# mdl.layers[2].set_weights(np.load("models/incident.npy"))


In [10]:

train_set = get_dataset(train_paths, train_dir, 1000)
val_set = get_dataset(val_paths, val_dir, 1000)

train_df = pd.DataFrame(train_set)
val_df = pd.DataFrame(val_set)

In [11]:
imgen = ImageDataGenerator(
    horizontal_flip=True,
    preprocessing_function=getpreprocessfunc(),
)

imgen = imgen.flow_from_dataframe(
    train_df,
    directory=train_dir,
    x_col="path",
    y_col=["incidents", "place"],
    weight_col=None,
    target_size=(256, 256),
    color_mode="rgb",
    classes=None,
    class_mode="multi_output",
    batch_size=64,
    shuffle=False,
    seed=True,
    save_to_dir=None,
    save_prefix="",
    save_format="png",
    subset=None,
    interpolation="nearest",
    validate_filenames=True,
)

Found 1000 validated image filenames.


In [12]:
losses = {
    "incidents_projection": weighted_loss,
    "places_projection": weighted_loss,
}

In [14]:
opt = keras.optimizers.Adam(lr=1e-5)

mdl = get_final_model()

mdl.compile(optimizer=opt, loss=losses, metrics=[get_weighted_accuracy()])

In [13]:
x, y = next(imgen)
o = mdl(x)

<tf.Tensor: shape=(64, 1024), dtype=float32, numpy=
array([[0.47994307, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [1.6543304 , 0.        , 0.        , ..., 0.        , 0.        ,
        0.17853671],
       [1.3494209 , 0.        , 1.6460733 , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [1.2620056 , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [1.7307619 , 0.        , 0.        , ..., 0.        , 0.        ,
        1.3992574 ],
       [0.46037802, 0.        , 0.65531677, ..., 0.        , 0.        ,
        0.        ]], dtype=float32)>

In [60]:
# itr = tqdm(imgen)
for x, y in imgen:
#     print(type(x), type(y))
    y1 = y[0]
    y2 = y[1]
    bs = y1.shape[0]
#     print(, "\r")
    out = mdl.predict(x)
    print(tf.argmax(out[0], axis=1))
    print(tf.argmax(out[1], axis=1))
#     print([y1.reshape(bs, -1), y2.reshape(bs, -1)])
    print(np.argmax(y1[:,0,:], axis=1))
    print(np.argmax(y2[:,0,:], axis=1))
    break

tf.Tensor(
[11  3 28  0 13 17  1 34  4 14 24  1 31 35 20 23 27 31 14 14 27  1 19 30
 28  3 10 38  4  8 40 35 27 31  8 16  7 10 38 41  7 35 37 37 35 27 31 28
 16  7 24 22 10 34 24 10 31 23 41 27 28 37  8  2], shape=(64,), dtype=int64)
tf.Tensor(
[49 49 49 49 49 49 39 21 11 49 49 24 20  4 49 49 49 13 49 16 37 49 25 49
 49 49 49 49 11  4 10 49 37 13 49 45 49 49 49  9 49 49 49 49  4 49 49 49
 49 49 49 20 49 45 48 49 20 49  1 37 49 44 15 49], shape=(64,), dtype=int64)
[ 0  0  0  0  0  0  1 34  4  0  0  1 31 35  0  0  0 31  0 14 27  0 19  0
  0  0  0  0  4  8 33  0 27 31  0 16  0  0  0 41  7  0  0  0 35  0  0  0
  0  0  0 22  0 34 24  0 31  0 41 27  0 37  8  0]
[49 49 49 49 49 49  0  0  0 49 49 24 20  0 49 49 49  0 49  0 37 49  0 49
 49 49 49 49  0  0 10 49 37 13 49  0 49 49 49  0  0 49 49 49  4 49 49 49
 49 49 49  0 49  0 48 49 20 49  1  0 49 44  0 49]
